<a href="https://colab.research.google.com/github/rcshivaji/hello-world/blob/master/Tweet_Fetch%7CClean%7CEncode%7CVectorize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [0]:
from keras.layers import Embedding
from keras.models import Sequential
import keras as K
import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
import re

#TWITTER AUTHENTICATION AND CLEANING CLASS

In [0]:
ACCESS_TOKEN = "2968868438-D3DI4UbBtB1QqUS51mwisuqSJfQy2gQgJQZV2oz"
ACCESS_TOKEN_SECRET = "yd9skWbmskRUHl5m3EhlC22TFqbnv6uMVj3FAwgbznrmV"
CONSUMER_KEY = "EQM191aq4lCbOOd3Sd0WT5NfU"
CONSUMER_SECRET = "cI96K1FRmrh3zt6rEQmM1xvDHv4XCQGQx37fscvUnDhCKPs8ST"

In [0]:
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df
    
def remove_punct(text):
    stp='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    text  = "".join([char for char in text if char not in stp])
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[A-Z]+', '', text)
    text = re.sub('[a-z]+', '', text)
    return text

#Tweet Fetching

In [0]:
twitter_client = TwitterClient()
tweet_analyzer = TweetAnalyzer()
api = twitter_client.get_twitter_client_api()
tweets = api.user_timeline(screen_name="aajtak", count=100)
df = tweet_analyzer.tweets_to_data_frame(tweets)
df2 = df[['tweets']].copy() 
df2['tweets'] = df2['tweets'].apply(lambda x: remove_punct(x))

#Dataframe to List

In [0]:
docs=df2['tweets'].values.tolist()

#One Hot Encoding

In [11]:
vocab_size = 100

encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[90, 71, 91, 28, 28, 37, 74, 75, 26, 38, 61, 27, 65], [85, 58, 74, 87, 19, 47, 57, 32, 50, 9, 62, 81, 38, 99, 88, 32], [89, 56, 75, 9, 10, 29, 23, 22, 48, 4, 83, 76], [41, 52, 32, 71, 11, 56, 26, 74, 41, 39, 63, 58], [25, 89, 29, 48, 65, 10, 37], [77, 1, 16, 47, 9, 71, 90, 9, 49, 95, 30, 39, 65, 31, 64, 32], [47, 71, 13, 67, 42, 5, 74, 87, 93, 5, 41, 5, 56, 19, 13, 32, 56, 37, 68, 3, 32, 58], [93, 71, 60, 47, 14, 51, 40, 71, 41, 6, 77, 16, 28, 17], [45, 41, 1, 89, 71, 91, 98, 14, 1, 90, 9, 62, 14, 1, 51, 39, 65, 88, 98], [2, 37, 71, 63, 40, 41, 47, 90, 48, 14, 52, 55, 28], [47, 71, 97, 9, 14, 40, 75, 33, 68, 31, 89], [41, 52, 32, 71, 11, 56, 26, 60, 86, 9, 54, 17, 56, 63, 60, 40, 23], [43, 59, 23, 96, 71, 67, 39, 90, 14], [48, 94, 48, 52, 48, 40, 74, 76, 90, 75, 42, 4, 50], [60, 86, 9, 75, 13, 31, 29, 58], [47, 71, 56, 61, 9, 87, 41, 75, 58, 94, 49, 28, 56, 79, 39, 24, 19, 73, 38, 88, 60, 41, 95, 99, 37], [90, 39, 61, 41, 85, 47, 31, 29, 58], [27, 80, 3, 48, 90, 61, 81, 20, 74, 27, 26

#Word2Vec

In [13]:
embedding_length = 5
max_doc_len = 25

encoded_docs = pad_sequences(encoded_docs, truncating='post', padding='post', maxlen=max_doc_len)

print(encoded_docs)

[[90 71 91 ...  0  0  0]
 [85 58 74 ...  0  0  0]
 [89 56 75 ...  0  0  0]
 ...
 [93 41 49 ...  0  0  0]
 [47  9 71 ...  0  0  0]
 [86 75 74 ...  0  0  0]]


In [14]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_length, input_length=max_doc_len))
model.compile('rmsprop', 'mse')
output = model.predict(encoded_docs)
print(output)











[[[ 0.02963746  0.00740267 -0.0124017   0.03801087 -0.01191514]
  [-0.02571554 -0.00575656  0.01095574 -0.03373845 -0.00849838]
  [ 0.013885    0.02403799 -0.00815298 -0.01298523 -0.01826804]
  ...
  [-0.02928478 -0.02179407  0.04947067  0.02874852  0.04958094]
  [-0.02928478 -0.02179407  0.04947067  0.02874852  0.04958094]
  [-0.02928478 -0.02179407  0.04947067  0.02874852  0.04958094]]

 [[-0.01395679 -0.00754518 -0.03229846  0.01345373  0.03898514]
  [ 0.00929401 -0.00245354 -0.04599508  0.03861643 -0.00033432]
  [-0.04235762 -0.03018929  0.02799929 -0.01558506 -0.04126792]
  ...
  [-0.02928478 -0.02179407  0.04947067  0.02874852  0.04958094]
  [-0.02928478 -0.02179407  0.04947067  0.02874852  0.04958094]
  [-0.02928478 -0.02179407  0.04947067  0.02874852  0.04958094]]

 [[ 0.01739441  0.02108803  0.01516994 -0.03057354  0.04203427]
  [ 0.02304814  0.01143237  0.03046841 -0.03755448  0.04999662]
  [-0.02362751  0.01549535 -0.01419044  0.044556   -0.00540794]
  ...
  [-0.02